In [1]:
import os
import pandas as pd

# Define the directory where your CSV files are stored
csv_dir = "../local/21_10/results"


In [ ]:
# Step 1: Load CSV files and extract relevant information
for csv_file in os.listdir(csv_dir):
    if csv_file.endswith(".csv"):

        csv_file_name = os.path.splitext(csv_file)[0]

        if len(csv_file_name.split("_")) == 3:
            task, model, subject = csv_file_name.split("_")
        else:
            task, _, model, subject = csv_file_name.split("_")
            
        df = pd.read_csv(os.path.join(csv_dir, csv_file))

In [ ]:
# # Step 3: Organize the metrics data
# def extract_metric(df, metric_name):
#     # Look for columns starting with 'val/' or 'mean_val/'
#     relevant_columns = [col for col in df.columns if col.startswith(('val/', 'mean_val/'))]
#     metric_columns = [col for col in relevant_columns if metric_name in col]
#     return df[metric_columns]

In [ ]:
# def extract_metric(df, metric_name):
#     relevant_columns = [col for col in df.columns if col.startswith(('val/', 'mean_val/'))]
#     metric_columns = [col for col in relevant_columns if metric_name in col]
#     return df[metric_columns]

In [ ]:

data_lr = {}
data_2D = {}

# Function to process CSV data
def process_csv_data(csv_data):
    # Assuming data_type1 and data_type2 are the strings containing your respective CSV data.

    # For type 1
    type1_df = pd.read_csv(pd.compat.StringIO(csv_data), header=None, names=['column'])

    # Extract mean_test and mean_val columns
    type1_df = type1_df[type1_df['column'].str.startswith(('mean_test', 'mean_val'))]

    # Split column into original column name and dictionary
    type1_df[['original_column', 'dictionary']] = type1_df['column'].str.split(',', expand=True)

    # Extract values from the dictionary
    type1_df = pd.concat([type1_df, type1_df['dictionary'].apply(lambda x: pd.Series(eval(x)))], axis=1)

    # Drop unnecessary columns
    type1_df = type1_df.drop(columns=['column', 'dictionary'])

    return type1_df

# Step 1: Load CSV files and extract relevant information
for csv_file in os.listdir(csv_dir):
    if csv_file.endswith(".csv"):

        csv_file_name = os.path.splitext(csv_file)[0]

        if len(csv_file_name.split("_")) == 3:
            task, model, subject = csv_file_name.split("_")
        else:
            task, _, model, subject = csv_file_name.split("_")
            
        
        
        df = pd.read_csv(os.path.join(csv_dir, csv_file))
        
        if 
        
        
        # df.columns = df.columns.str.replace('mean_', '')  # Remove "mean_" prefix

        # Step 2: Separate LR and 2D data
        if task == "LR":
            if model in data_lr:
                data_lr[model].append(df.assign(subject_name=subject))
            else:
                data_lr[model] = [df.assign(subject_name=subject)]
        if task == "2D":
            if model in data_2D:
                data_2D[model].append(df.assign(subject_name=subject))
            else:
                data_2D[model] = [df.assign(subject_name=subject)]



In [ ]:
# Process LR Data
for model, dfs in data_lr.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df = combined_df[combined_df['subject_name'].notna()]
    acc_lr_data = combined_df.filter(like='acc').set_index('subject_name').drop_duplicates()
    f1_lr_data = combined_df.filter(like='f1').set_index('subject_name').drop_duplicates()

    # Further processing as needed

# Process 2D Data
for model, dfs in data_2D.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df = combined_df[combined_df['subject_name'].notna()]
    acc_2D_data = combined_df.filter(like='acc').set_index('subject_name').drop_duplicates()
    f1_2D_data = combined_df.filter(like='f1').set_index('subject_name').drop_duplicates()

    # Further processing as needed

# Step 3: Process CSV data (assuming you have data_type1 and data_type2)
data_type1 = "your_csv_data_for_type1_here"
data_type2 = "your_csv_data_for_type2_here"

type1_df = process_csv_data(data_type1)
type2_df = process_csv_data(data_type2)

# Renaming the columns
type1_df = type1_df.rename(columns=lambda x: f'mean_{x}')
type2_df = type2_df.rename(columns=lambda x: f'mean_{x}')

# Combining both dataframes (if needed)
combined_df = pd.concat([type1_df, type2_df], axis=1)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Step 3: Process the data
# Initialize an empty dictionary to store DataFrames for each model
# Define the directory where your CSV files are stored
csv_dir = "../local/21_10/results"

data_lr = {}
data_2D = {}

# Step 1: Load CSV files and extract relevant information
for csv_file in os.listdir(csv_dir):
    if csv_file.endswith(".csv"):

        csv_file_name = os.path.splitext(csv_file)[0]
        # print(csv_file_name)
        # print(csv_file_name.split("_"))

        if len(csv_file_name.split("_")) == 3:
            task, model, subject = csv_file_name.split("_")
        else:
            task, _, model, subject = csv_file_name.split("_")
        
        df = pd.read_csv(os.path.join(csv_dir, csv_file))
        df.columns = df.columns.str.replace('mean_', '')  # Remove "mean_" prefix

        # Step 2: Separate LR and 2D data
        if task == "LR":
            if model in data_lr:
                data_lr[model].append(df.assign(subject_name=subject))
            else:
                data_lr[model] = [df.assign(subject_name=subject)]
        if task == "2D":
            if model in data_2D:
                data_2D[model].append(df.assign(subject_name=subject))
            else:
                data_2D[model] = [df.assign(subject_name=subject)]

def extract_metric(df, metric_name):
    if 'mean_' + metric_name in df.columns:
        return df[df['subject_name'].notna()][['subject_name', 'mean_' + metric_name]]
    else:
        return df[df['subject_name'].notna()]['subject_name'].apply(lambda x: x[1:-1][metric_name]).reset_index(drop=True)

# Example usage
acc_lr_data = {}
f1_lr_data = {}
acc_2D_data = {}
f1_2D_data = {}

# Step 3: Process the data
for model, dfs in data_lr.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    acc_lr_data[model]  = extract_metric(combined_df, 'acc').set_index('subject_name').drop_duplicates()
    f1_lr_data[model] = extract_metric(combined_df, 'f1').set_index('subject_name').drop_duplicates()

for model, dfs in data_2D.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    acc_2D_data[model]  = extract_metric(combined_df, 'acc').set_index('subject_name').drop_duplicates()
    f1_2D_data[model] = extract_metric(combined_df, 'f1').set_index('subject_name').drop_duplicates()


In [ ]:
# def extract_metric(df, metric_name):
#     print(df)
#     if 'mean_' + metric_name in df.columns:
#         relevant_columns = [col for col in df.columns if col.startswith(('mean_val/'))]
#         metric_columns = [col for col in relevant_columns if metric_name in col]
#         return df[['subject_name'] + metric_columns]   # Include 'subject' column in the result
    

In [ ]:
acc_lr_data = {}
f1_lr_data = {}
acc_2D_data = {}
f1_2D_data = {}

# Step 3: Process the data
for model, dfs in data_lr.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    acc_lr_data[model]  = extract_metric(combined_df, 'acc').set_index('subject_name').drop_duplicates()
    f1_lr_data[model] = extract_metric(combined_df, 'f1').set_index('subject_name').drop_duplicates()
    
    
for model, dfs in data_2D.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    acc_2D_data[model]  = extract_metric(combined_df, 'acc').set_index('subject_name').drop_duplicates()
    f1_2D_data[model] = extract_metric(combined_df, 'f1').set_index('subject_name').drop_duplicates()

In [ ]:
f1_2D_data

In [ ]:
acc_2D_data["csp"]

In [ ]:
# Assuming 'subject_name' is the column that contains subject names
# acc_lr_data = [(model, extract_metric(pd.concat([df.assign(subject_name=subject) for model, subject, df in task_LR_data if model == model], ignore_index=True), 'acc').set_index('subject_name').drop_duplicates()) for model, _, _ in task_LR_data]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming acc_lr_data is a dictionary where keys are model names and values are DataFrames
plt.figure(figsize=(12, 6))

x_positions = np.arange(len(acc_lr_data))

for i, (model, df) in enumerate(acc_lr_data.items()):
    avg_acc = df[['val/acc']].mean(axis=1, skipna=True)
    
    # Adjust the x position for each boxplot
    plt.boxplot(avg_acc.values, positions=[i])

    
plt.title('Boxplot for Model Performance LR task')
plt.xlabel('Model')
plt.ylabel('Average Accuracy')
plt.xticks(x_positions, [model for model, _ in acc_lr_data.items()], rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming acc_lr_data is a dictionary where keys are model names and values are DataFrames
plt.figure(figsize=(12, 6))

x_positions = np.arange(len(acc_2D_data))

for i, (model, df) in enumerate(acc_2D_data.items()):
    avg_acc = df[['val/acc']].mean(axis=1, skipna=True)
    
    # Adjust the x position for each boxplot
    plt.boxplot(avg_acc.values, positions=[i])

    
plt.title('Boxplot for Model Performance 2D task')
plt.xlabel('Model')
plt.ylabel('Average Accuracy')
plt.xticks(x_positions, [model for model, _ in acc_2D_data.items()], rotation=45)
plt.show()

In [ ]:
!pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

# Assuming acc_lr_data is a dictionary where keys are model names and values are DataFrames
plt.figure(figsize=(12, 6))

# Create lists to store subject names and average accuracies
subject_names = []
avg_accuracies = []

# Iterate through the acc_lr_data dictionary
for i, (model, df) in enumerate(acc_lr_data.items()):
    avg_acc = df[['val/acc']].mean(axis=1, skipna=True)
    
    # Add subject names and corresponding average accuracies
    subject_names.extend([model] * len(avg_acc))
    avg_accuracies.extend(avg_acc)
    
    # Adjust the x position for each boxplot
    plt.boxplot(avg_acc.values, positions=[i])

# Add Gaussian distribution plot
x = np.linspace(min(avg_accuracies), max(avg_accuracies), 100)
pdf = stats.norm.pdf(x, np.mean(avg_accuracies), np.std(avg_accuracies))
plt.plot(x, pdf, color='red')

# Add subject names as dots
plt.scatter(range(len(subject_names)), avg_accuracies, c='black', marker='o', s=10)

plt.title('Model Performance')
plt.xlabel('Model')
plt.ylabel('Average Accuracy')
plt.xticks(range(len(acc_lr_data)), [model for model, _ in acc_lr_data.items()], rotation=45)
plt.show()
